In [1]:
import cv2
import mediapipe as mp
import random

# Initialize mediapipe for hand detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Set up camera capture
cap = cv2.VideoCapture(0)

# Game moves
moves = ['rock', 'paper', 'scissors']

def get_computer_move():
    return random.choice(moves)

# Function to detect hand gesture (rock, paper, or scissors)
def get_hand_move(hand_landmarks):
    if hand_landmarks:
        fingers = []
        # Thumb
        if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
            fingers.append(1)  # Open thumb
        else:
            fingers.append(0)  # Closed thumb

        # Four fingers
        for id in range(8, 21, 4):
            if hand_landmarks.landmark[id].y < hand_landmarks.landmark[id - 2].y:
                fingers.append(1)  # Open finger
            else:
                fingers.append(0)  # Closed finger

        # Determine move
        if fingers == [0, 0, 0, 0, 0]:
            return 'rock'
        elif fingers == [0, 1, 1, 1, 1]:
            return 'paper'
        elif fingers == [0, 1, 1, 0, 0]:
            return 'scissors'

    return None

# Function to check winner and update score
def check_winner(player_move, computer_move, scores):
    if player_move == computer_move:
        return 'Tie'
    elif (player_move == 'rock' and computer_move == 'scissors') or \
         (player_move == 'scissors' and computer_move == 'paper') or \
         (player_move == 'paper' and computer_move == 'rock'):
        scores['player'] += 1
        return 'Player Wins'
    else:
        scores['computer'] += 1
        return 'Computer Wins'

# Function to show countdown on screen
def show_countdown(frame, countdown):
    # Clear previous text by filling the frame with black
    frame[:] = (0, 0, 0)
    cv2.putText(frame, f'Next Round in: {countdown}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
    cv2.imshow('Rock Paper Scissors', frame)

# Initialize scores
scores = {'player': 0, 'computer': 0}

# Main loop
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip frame to act like a mirror

    # Convert BGR to RGB for mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    player_move = None
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            player_move = get_hand_move(hand_landmarks)

    # If a player's move is detected
    if player_move:
        # Computer makes a move
        computer_move = get_computer_move()
        cv2.putText(frame, f'Player Move: {player_move}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Computer Move: {computer_move}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Determine the winner and update score
        result = check_winner(player_move, computer_move, scores)
        cv2.putText(frame, result, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show scores
        cv2.putText(frame, f'Player Score: {scores["player"]}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Computer Score: {scores["computer"]}', (10, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Show the result and countdown before the next round
        cv2.imshow('Rock Paper Scissors', frame)
        cv2.waitKey(3000)  # Wait 3 seconds to display the result

        # Countdown before the next round
        for countdown in range(3, 0, -1):
            show_countdown(frame, countdown)
            cv2.waitKey(1000)  # Wait 1 second

    else:
        cv2.putText(frame, 'Show Rock, Paper, or Scissors', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show scores even when no hand is detected
    cv2.putText(frame, f'Player Score: {scores["player"]}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Computer Score: {scores["computer"]}', (10, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('Rock Paper Scissors', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
hands.close()

C:\Users\jeeva\ana\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
